In [263]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot
import seaborn as sns
import pydot

from sklearn import preprocessing
from sklearn.externals.six import StringIO  
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.cross_validation import train_test_split

In [184]:
normalized_columns = [
    'eloadas_latogatas',
    'jegyzet_eloadason',
    'gyakorlat_latogatas',
    'mibol_tanul',
    'egyedul_vagy_csoportosan',
    'jegyzet_tanulashoz',
    'ajanlott_olvasas',
    'zh_keszules',
    'vizsgara_keszules',
    'tanulasi_napszak',
    'szamonkeres_elott_tanulni',
    'zenehalgatas',
    'lakhely',
    'megoldas_reprodukalasa',
    'elert_kredit'
]

ordinal_transformations = {
    u'Soha': 0,
    u'Néha': 1,
    u'Rendszeresen': 2,
    u'Nem': 0,
    u'Igen': 1,
    u'0 - 10 kredit': 0,
    u'11 - 15 kredit': 1, 
    u'16 - 20 kredit': 2,
    u'21 - 25 kredit': 3, 
    u'26 - 30 kredit': 4, 
    u'31 - 37 kredit': 5,
    u'38 vagy annál több kredit': 6
}

ordinal_cols = [
    'eloadas_latogatas',
    'jegyzet_eloadason',
    'gyakorlat_latogatas',
    'jegyzet_tanulashoz',
    'ajanlott_olvasas',
    'szamonkeres_elott_tanulni',
    'zenehalgatas',
    'megoldas_reprodukalasa',
]
nominal_cols = [
    'mibol_tanul',
    'egyedul_vagy_csoportosan',
    'zh_keszules',
    'vizsgara_keszules',
    'tanulasi_napszak',
    'lakhely',
]
target_col = ['elert_kredit']

In [352]:
kerdo_iv = pd.read_csv("kerdo_iv.csv")
kerdo_iv = kerdo_iv[kerdo_iv.columns[1:]]
kerdo_iv.columns = normalized_columns

In [353]:
kerdo_iv.head()

,eloadas_latogatas,jegyzet_eloadason,gyakorlat_latogatas,mibol_tanul,egyedul_vagy_csoportosan,jegyzet_tanulashoz,ajanlott_olvasas,zh_keszules,vizsgara_keszules,tanulasi_napszak,szamonkeres_elott_tanulni,zenehalgatas,lakhely,megoldas_reprodukalasa,elert_kredit
0,Néha,Nem,Rendszeresen,Diasorok,Csoportosan,Néha,Nem,Pár nappal,1 héttel,Délután,Rendszeresen,Néha,Albérlet,Néha,26 - 30 kredit
1,Néha,Igen,Rendszeresen,Mások által készített jegyzetből,Egyedül,Néha,Igen,Pár nappal,1 héttel,Éjszaka / este,Rendszeresen,Rendszeresen,Kollégiumban,Rendszeresen,0 - 10 kredit
2,Néha,Nem,Rendszeresen,Diasorok,Egyedül,Néha,Nem,1 héttel,1 héttel,Délután,Rendszeresen,Néha,Albérlet,Soha,26 - 30 kredit
3,Néha,Nem,Néha,VikWikis anyag,Csoportosan,Néha,Nem,Pár nappal,1 héttel,Éjszaka / este,Rendszeresen,Rendszeresen,Kollégiumban,Néha,31 - 37 kredit
4,Néha,Nem,Néha,VikWikis anyag,Egyedül,Néha,Nem,1 héttel,1 héttel,Éjszaka / este,Rendszeresen,Rendszeresen,Otthon,Néha,26 - 30 kredit


In [354]:
for column in ordinal_cols+target_col:
    kerdo_iv[column] = kerdo_iv[column].apply(lambda x: transformations[x.decode('utf-8')])

In [358]:
kerdo_iv.head(10)

,eloadas_latogatas,jegyzet_eloadason,gyakorlat_latogatas,mibol_tanul,egyedul_vagy_csoportosan,jegyzet_tanulashoz,ajanlott_olvasas,zh_keszules,vizsgara_keszules,tanulasi_napszak,szamonkeres_elott_tanulni,zenehalgatas,lakhely,megoldas_reprodukalasa,elert_kredit
0,1,0,2,Diasorok,Csoportosan,1,0,Pár nappal,1 héttel,Délután,2,1,Albérlet,1,4
1,1,1,2,Mások által készített jegyzetből,Egyedül,1,1,Pár nappal,1 héttel,Éjszaka / este,2,2,Kollégiumban,2,0
2,1,0,2,Diasorok,Egyedül,1,0,1 héttel,1 héttel,Délután,2,1,Albérlet,0,4
3,1,0,1,VikWikis anyag,Csoportosan,1,0,Pár nappal,1 héttel,Éjszaka / este,2,2,Kollégiumban,1,5
4,1,0,1,VikWikis anyag,Egyedül,1,0,1 héttel,1 héttel,Éjszaka / este,2,2,Otthon,1,4
5,2,1,2,VikWikis anyag,Egyedül,1,1,1 héttel,1 héttel,Éjszaka / este,1,1,Albérlet,1,4
6,1,1,1,Diasorok,Egyedül,1,1,Pár nappal,Pár nappal,Éjszaka / este,2,1,Otthon,2,5
7,1,1,2,Előző évi zhk/vizsgák tanulmányozása és megoldása,Egyedül,1,0,Pár nappal,1 héttel,Éjszaka / este,2,2,Otthon,2,4
8,2,1,2,Előző évi zhk/vizsgák tanulmányozása és megoldása,Csoportosan,1,0,1 héttel,1 héttel,Délután,2,1,Albérlet,2,2
9,2,1,2,Előző évi zhk/vizsgák tanulmányozása és megoldása,Csoportosan,2,1,1 héttel,1 héttel,Éjszaka / este,2,1,Albérlet,1,5


# Nominálisok átírása

In [199]:
class ItemSelector(BaseEstimator, TransformerMixin):

    def __init__(self, key, transpose=False):
        self.key = key
        self.transpose = transpose

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X[self.key].values
        if self.transpose:
            return X[:, np.newaxis]
        return X
    
    
class ItemsToDictorizer(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return [{self.key: val} for val in X]

    
class IdentityTransformer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X

# Modell betanítása és a Data és Target adat létrehozása

In [200]:
target = kerdo_iv[target_col].values
target.shape

(141, 1)

In [201]:
def create_pipe(column, nominal):
    if nominal:
        return Pipeline([
            ('sel', ItemSelector(column)),
            ('dict', ItemsToDictorizer(column)),
            ('trans', DictVectorizer(sparse=False))
        ])
    return Pipeline([
        ('sel', ItemSelector(column, True)),
        ('trans', IdentityTransformer())
    ])

In [211]:
transformers = ([(col, create_pipe(col, True)) for col in nominal_cols] + 
                [(col, create_pipe(col, False)) for col in ordinal_cols])

In [348]:
feat = FeatureUnion(transformer_list=transformers)
cls = DecisionTreeClassifier(max_depth=8)
pipe = Pipeline([
    ('feat', feat),
    ('cls', cls)
])

In [359]:
pipe.fit(kerdo_iv, target)

Pipeline(steps=[('feat', FeatureUnion(n_jobs=1,
       transformer_list=[('mibol_tanul', Pipeline(steps=[('sel', ItemSelector(key='mibol_tanul', transpose=False)), ('dict', ItemsToDictorizer(key='mibol_tanul')), ('trans', DictVectorizer(dtype=<type 'numpy.float64'>, separator='=', sort=True,
        sparse=F...min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best'))])

# Döntési fa létrehozása és kimentése pdf-be

In [246]:
def get_feature_names(index):
    return feat.transformer_list[index][-1].steps[-1][-1].feature_names_

In [259]:
feature_names =  ([get_feature_names(index) for index in xrange(len(nominal_cols))]
                  + [[col] for col in ordinal_cols])
feature_names = [item for sublist in feature_names for item in sublist]

In [262]:
dot_data = StringIO()
export_graphviz(cls, feature_names=feature_names, out_file=dot_data) 
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
graph.write_pdf("tree.pdf") 

True

---

# Train és Test adat létrehozása

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(kerdo_iv, target, random_state=42,train_size = 0.7)

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
pipe.score(X_test,y_test)